In [6]:
import re

def load_pokemon_details(file_path="pokemon_data_cleaned.txt"):
    pokemon_data = {}
    with open(file_path, "r", encoding="utf-8") as file:
        data = file.read().strip().split("---")  # Séparer chaque Pokémon par "---"
        
    for entry in data:
        lines = entry.strip().split("\n")
        if not lines:
            continue

        # Extraire le nom du Pokémon
        match = re.match(r"Pokémon:\s*(.+)", lines[0])
        if match:
            pokemon_name = match.group(1).strip()
            pokemon_data[pokemon_name] = "\n".join(lines[1:])  # Stocker le reste des infos

    return pokemon_data


In [8]:
import random
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re

# Charger le modèle fine-tuné
MODEL_PATH = "fine_tuned_gpt2-3"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_PATH)
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)

# Fonction pour générer une équipe Pokémon
def generate_team(prompt="Donne-moi une équipe format UU:", max_length=100):

    inputs = tokenizer(prompt, return_tensors="pt")
    
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_beams=3,       # Désactivation du beam search pour plus de diversité
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        temperature=0.7,   # Augmenter la température pour plus de diversité
        pad_token_id=tokenizer.pad_token_id,
    )

    # Décoder l'output du modèle
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraction des Pokémon en prenant tout ce qui suit le ":"
    pokemon_part = decoded_output.split(":")[-1]  # Prendre la partie après ":"
    
    # Utilisation d'une regex pour extraire proprement les noms des Pokémon
    pokemons = re.findall(r"\b[A-Za-z-]+\b", pokemon_part)

    # Vérifier si nous avons moins de 6 Pokémon et ajouter des Pokémon fictifs si nécessaire
    while len(pokemons) < 6:
        pokemons.append("Pikachu")  # ou n'importe quel autre Pokémon de remplacement

    return ", ".join(pokemons[:6])  # Limiter à 6 Pokémon

# Tester la génération
#print(generate_team())


In [9]:
def generate_team_with_details():
    team = generate_team()  # Générer une équipe
    pokemon_details = load_pokemon_details()  # Charger les détails des Pokémon
    
    print("\n**Équipe générée :**\n")
    for pokemon in team.split(", "):  # Assurez-vous que l'équipe est une liste séparée par des virgules
        print(f"- {pokemon}")
    
    print("\n**Détails des Pokémon dans l'équipe :**\n")
    for pokemon in team.split(", "):  # Assurez-vous que l'équipe est une liste séparée par des virgules
        if pokemon in pokemon_details:
            print(f"{pokemon} :\n{pokemon_details[pokemon]}\n")
        else:
            print(f"{pokemon} : Aucun détail trouvé.\n")

# Exécuter la génération avec détails
generate_team_with_details()



**Équipe générée :**

- Weavile
- Excadrill
- Hippowdon
- Weezing-Galar
- Slowbro
- Urshifu-Rapid-Strike

**Détails des Pokémon dans l'équipe :**

Weavile :
- Moves: Icicle Crash, Knock Off, Beat Up/Ice Shard, Beat Up/Pursuit
- Item: Choice Band
- Ability: Pressure
- Nature: Jolly
- EVs: 252 Atk/4 SpD/252 Spe/
- Format: OU/Monotype/Battle Spot Singles/CAP/Draft/Mix and Mega/1v1/

Excadrill :
- Moves: Stealth Rock, Earthquake, Toxic/Rock Tomb, Rapid Spin
- Item: Focus Sash
- Ability: Mold Breaker
- Nature: Jolly
- EVs: 252 Atk/4 SpD/252 Spe/
- Format: OU/AG/Uber/Monotype/Doubles/Battle Spot Singles/VGC18/1v1/

Hippowdon :
- Moves: Stealth Rock, Earthquake, Slack Off, Whirlwind/Toxic
- Item: Rocky Helmet
- Ability: Sand Force
- Nature: Impish
- EVs: 252 HP/80 Def/176 SpD/
- Format: UU/OU/Monotype/Battle Spot Singles/

Weezing-Galar : Aucun détail trouvé.

Slowbro :
- Moves: Scald, Psyshock, Slack Off, Ice Beam
- Item: Colbur Berry
- Ability: Regenerator
- Nature: Bold
- EVs: 252 HP/252 